<font color = #09bbff size = 5 face="標楷體"> 匯入函式庫(模組) </font>

In [1]:
#爬蟲相關套件
import requests
from bs4 import BeautifulSoup
import time

#資料處理與儲存相關套件
import pandas as pd
import numpy as np
import re, random, time, os

#------------------------------------------------
#Not every package has a __version__ attribute.
import sys
print('python version = ', sys.version)
from importlib.metadata import version
print('requests version = ', version('requests')) #same as (requests.__version__)
print('pandas version = ', pd.__version__)


python version =  3.9.16 (main, Jan 11 2023, 16:16:36) [MSC v.1916 64 bit (AMD64)]
requests version =  2.29.0
pandas version =  1.5.3


<font color= #00e9de size= 5 face="標楷體"> 定義函式，供使用者輸入欲爬取之主題與頁數 </font>

In [2]:
def user_demand():
    theme = input('請輸入欲爬取的目標看板: ')
    latest_page = eval(input('請輸入欲爬取之末頁頁碼: '))
    page =  eval(input('請輸入欲爬取的總頁數: '))
    
    return theme, latest_page, page

<font color= #00e9de size= 5 face="標楷體"> 定義函式，取得多頁 ptt 網址 </font>

In [3]:
#看板網址格式 https://www.ptt.cc/bbs/Soft_Job/index.html
#內文連結標籤元素: <a href="/bbs/Soft_Job/M.1677220493.A.D1C.html"

def multipage_ptt_urls(user_demand):
    partial_url = 'https://www.ptt.cc/bbs/'
    theme, latest_page, page = user_demand()
    start_page = latest_page - page +1
    print('爬取起始頁: ', start_page)
    ptt_urls = []  #設空串列儲存組合好的網址
    for p in range(start_page, latest_page+1):
        entire_url = f'{partial_url}{theme}/index{p}.html'
        ptt_urls.append(entire_url)
        
    return ptt_urls

<font color= #0fe9de size= 5 face="標楷體"> 定義函式向伺服器請求網站資料，爬取網頁 HTML </font>
* [requests 官方文件](https://requests.readthedocs.io/en/latest/api/)
* [BeautifulSoup 官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [4]:
def fetch_html(url_list = multipage_ptt_urls):  
    # 設定表頭偽裝成瀏覽器。(敘述太長以反斜線 '\' 分行方便閱讀)
    ptt_headers = {'over18':'1',          #一直向 server 回答滿 18 歲了 !
                    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84\
                    Safari/537.36'}
    if url_list == multipage_ptt_urls: 
        # 當參數為預設參數，調用 "multipage_ptt_urls 回呼函式"，回傳 ptt 多頁網址
        url_list = multipage_ptt_urls(user_demand)  #type(url_list): <class 'list'> 
    else:
        # 根據輸入之參數(串列)進行網頁請求
        url_list = url_list  
        
    bptree_dic = {}   #存放每頁網頁的 HTML 資料 
    for i in range(len(url_list)):    
        try:
            # 利用 requests 模組中 get 指令請求網站資料，並將網站 Response 回應物件指定為 r 變數
            r = requests.get(url_list[i], headers = ptt_headers)
            # 建立 BeautifulSoup 物件，並利用 'lxml'作為解析器(parser)，解析速度快相容性佳
            bptree_dic[i] = BeautifulSoup(r.text, 'lxml')  #將解析好的 html 結構樹存到字典中
        except Exception as err:
            print('程式出現異常:', err) #python 內建的異常處理訊息
            print(r.raise_for_status())#異常處理訊息(針對 4XX client error or 5XX server error response)，快速協助偵錯 
            print(r.reason)            #Textual reason of responded HTTP Status, e.g. “Not Found” or “OK”.
        else:
            # Response 物件中 "status_code" 為 http 狀態碼，顯示 200 表示請求成功，網頁資料成功被下載
            if r.status_code == requests.codes.ok:  
                print(f'{i+1}.成功取得 {url_list[i]} 資料')
            else:  
                print(f'{i+1}.{url_list[i]} 取得失敗')
         
        if i % 50 == 0:time.sleep(random.uniform(3,5))
            
    return bptree_dic

bp_dic = fetch_html()

請輸入欲爬取的目標看板: Soft_Job
請輸入欲爬取之末頁頁碼: 1700
請輸入欲爬取的總頁數: 10
爬取起始頁:  1691
1.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1691.html 資料
2.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1692.html 資料
3.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1693.html 資料
4.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1694.html 資料
5.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1695.html 資料
6.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1696.html 資料
7.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1697.html 資料
8.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1698.html 資料
9.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1699.html 資料
10.成功取得 https://www.ptt.cc/bbs/Soft_Job/index1700.html 資料


<font color= #0fe9de size= 5 face="標楷體">利用 BeautifulSoup 抓取每篇貼文資訊 </font>

In [5]:
post_divs = []    #存放每篇貼文 html 區塊
for a in bp_dic:    #一一取出字典中每頁 BeautifulSoup 資料  
    # print(type(bp_dic[a]))    #<class 'bs4.BeautifulSoup'>
    # 找出各篇貼文之 html 區塊
    post_divs += bp_dic[a].select('div.r-ent')   #<class 'bs4.element.ResultSet'>
    
posts_info = {}    #存放貼文資訊
article_links = []    #存放貼文連結網址
num = 0    #貼文編號
for ac in post_divs:    #一一取出各篇貼文之資訊
    num += 1
    title = ac.find('div', class_ = 'title').text.strip()    # 文章標題  <class 'bs4.element.Tag'>
    author = ac.find('div', class_ = 'author').text.strip()  # 作者
    posttime = ac.find('div', 'date').getText().strip()      # 發文時間
    href = ac.find("a")["href"]    #同 find('a').get('href')                         # 文章超連結
    push_num = ac.find('div', 'nrec').text                   # 推文數
    info_key = ['標題', '作者', '發文日期', '推文數', '文章超連結']
    info_value = [title, author, posttime, push_num, (f'https://www.ptt.cc{href}')]
    
    # 將各篇貼文之資訊存成字典型態
    info = {info_key[n]:info_value[n] for n in range(len(info_key))}
    posts_info[f'第 {num} 篇'] = info
    
    # 將各篇貼文之內文超連結存成 list, 以便後續利用 fetch_html(url_list) 函式發送請求
    article_links += [info['文章超連結']]
    
print(f'內文超連結數目: {len(article_links)} 項')    
posts_info

內文超連結數目: 200 項


{'第 1 篇': {'標題': '[請益] PATM frontend and data pipeline',
  '作者': 'quartics',
  '發文日期': '3/03',
  '推文數': '',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646319196.A.7D9.html'},
 '第 2 篇': {'標題': '[徵才] 清大孫民實驗室 誠徵專任研究助理',
  '作者': 'wuheiru',
  '發文日期': '3/04',
  '推文數': '4',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646323333.A.B7F.html'},
 '第 3 篇': {'標題': 'Re: [請益] 生物資訊助理 請益',
  '作者': 'syslabdt',
  '發文日期': '3/04',
  '推文數': '4',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646330940.A.D5C.html'},
 '第 4 篇': {'標題': '[徵才]統一證券誠徵2022資訊金融科技MA年薪百萬',
  '作者': 'weathers',
  '發文日期': '3/04',
  '推文數': '3',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646381809.A.D30.html'},
 '第 5 篇': {'標題': '[心得] 產品經理面試前準備分享',
  '作者': 'abc5555',
  '發文日期': '3/04',
  '推文數': '5',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646387859.A.083.html'},
 '第 6 篇': {'標題': '[請益] 金融業問題請教',
  '作者': 'Tenno081',
  '發文日期': '3/04',
  '推文數': '26',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1646398715.A.3C7.html'},
 '第 7 篇':

In [8]:
for ac in post_divs:
    href = ac.find("a")["href"]  
    print(href)

/bbs/Soft_Job/M.1646319196.A.7D9.html
/bbs/Soft_Job/M.1646323333.A.B7F.html
/bbs/Soft_Job/M.1646330940.A.D5C.html
/bbs/Soft_Job/M.1646381809.A.D30.html
/bbs/Soft_Job/M.1646387859.A.083.html
/bbs/Soft_Job/M.1646398715.A.3C7.html
/bbs/Soft_Job/M.1646460920.A.25A.html
/bbs/Soft_Job/M.1646461853.A.B08.html
/bbs/Soft_Job/M.1646470505.A.5D4.html
/bbs/Soft_Job/M.1646483246.A.73C.html
/bbs/Soft_Job/M.1646494112.A.099.html
/bbs/Soft_Job/M.1646539784.A.80E.html
/bbs/Soft_Job/M.1646558587.A.40A.html
/bbs/Soft_Job/M.1646573613.A.C7D.html
/bbs/Soft_Job/M.1646577648.A.546.html
/bbs/Soft_Job/M.1646650682.A.3BC.html
/bbs/Soft_Job/M.1646664881.A.469.html
/bbs/Soft_Job/M.1646721584.A.302.html
/bbs/Soft_Job/M.1646726363.A.D8F.html
/bbs/Soft_Job/M.1646790547.A.7AA.html
/bbs/Soft_Job/M.1646794195.A.DAF.html
/bbs/Soft_Job/M.1646806894.A.7AA.html
/bbs/Soft_Job/M.1646818282.A.0EC.html
/bbs/Soft_Job/M.1646839075.A.DBC.html
/bbs/Soft_Job/M.1646839109.A.6D0.html
/bbs/Soft_Job/M.1646909938.A.63C.html
/bbs/Soft_Jo

<font color= #0fe9de size= 5 face="標楷體"> 發送每篇貼文連結之請求，爬取內文文字資料</font>

In [6]:
links_bp_dic = fetch_html(url_list= article_links)

1.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1674717939.A.DDB.html 資料
2.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1674828041.A.5EB.html 資料
3.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1674909243.A.7C9.html 資料
4.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1674982234.A.5BE.html 資料
5.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1674995504.A.73E.html 資料
6.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675078503.A.71A.html 資料
7.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675091717.A.CA2.html 資料
8.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675182226.A.91E.html 資料
9.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675216698.A.7E5.html 資料
10.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675254894.A.CB1.html 資料
11.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675388013.A.EA8.html 資料
12.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675392784.A.BCA.html 資料
13.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675422268.A.8CA.html 資料
14.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675433282.A.283.html 資料
15.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1675443364.A.340.html 資料
16.成

124.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678755633.A.0F7.html 資料
125.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678767190.A.041.html 資料
126.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678777673.A.15D.html 資料
127.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678796032.A.BA4.html 資料
128.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678811866.A.D31.html 資料
129.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678856174.A.098.html 資料
130.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678928363.A.8F1.html 資料
131.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678945383.A.3EC.html 資料
132.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678957330.A.C92.html 資料
133.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678966172.A.079.html 資料
134.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1678983389.A.3F5.html 資料
135.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1679060469.A.435.html 資料
136.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1679072256.A.EF3.html 資料
137.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.1679123346.A.518.html 資料
138.成功取得 https://www.ptt.cc/bbs/Soft_Job/M.16791

<font color= #0fe9de size= 5 face="標楷體"> 抓取文章內文</font>

In [7]:
content_div = links_bp_dic[0].find('div', id = 'main-content')
content_list = list(content_div)  #將 html 結構樹轉為列表，list中元素的順序與 HTML 中元素出現的順序相同。
element_type = set(type(i) for i in content_list)
print(f'content_list element_type: {element_type}')
content_list

content_list element_type: {<class 'bs4.element.Tag'>, <class 'bs4.element.NavigableString'>}


[<div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">Lordaeron (Terry)</span></div>,
 <div class="article-metaline-right"><span class="article-meta-tag">看板</span><span class="article-meta-value">Soft_Job</span></div>,
 <div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[心得] Google裁員1.2萬人只寄電郵</span></div>,
 <div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Thu Jan 26 15:25:37 2023</span></div>,
 '\n來源\n',
 <a href="https://udn.com/news/story/7086/6930865?from=udn_ch2_menu_v2_main_index" rel="noreferrer noopener nofollow" target="_blank">https://udn.com/news/story/7086/6930865?from=udn_ch2_menu_v2_main_index</a>,
 '\nGoogle裁員1.2萬人只寄電郵 20年工程師「直接被封鎖」\n在Google擔任研究員的拉塞爾（Dan Russell）則說，「當我凌晨4點完成一項重要分析，\n我要上班時發現鑰匙卡壞了，我在Google工作17年半。\n\n\n這就是台灣公司和google 的實力的差異了。\n不用交接!! 一家可以直接吊打全台灣所有公司。\n\n--\n',
 <span class="f2">※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 

In [8]:
# 檢查貼文篇數與內文連結數量是否相對應
if num == len(article_links):    
    for l in links_bp_dic:     #一一取出每篇貼文的 html 結構樹
        posts_info[f'第 {l+1} 篇']['內文'] = ''
        content_div = links_bp_dic[l].find('div', id = 'main-content')
        content_list = list(content_div)  #將 html 結構樹轉為列表
        for item in content_list:
            if isinstance(item, str):
                posts_info[f'第 {l+1} 篇']['內文'] += f'{item.strip()}'
            elif item.has_attr('href'):
                posts_info[f'第 {l+1} 篇']['內文'] += f'{item.text}' 
        print(f"第 {l+1} 篇:{posts_info[f'第 {l+1} 篇']['內文']}")
                           
else:print('貼文數量與貼文連結數量不一致')

第 1 篇:來源https://udn.com/news/story/7086/6930865?from=udn_ch2_menu_v2_main_indexGoogle裁員1.2萬人只寄電郵 20年工程師「直接被封鎖」
在Google擔任研究員的拉塞爾（Dan Russell）則說，「當我凌晨4點完成一項重要分析，
我要上班時發現鑰匙卡壞了，我在Google工作17年半。


這就是台灣公司和google 的實力的差異了。
不用交接!! 一家可以直接吊打全台灣所有公司。

--
第 2 篇:追了一天動漫來回點東西，
轉生公主天才千金好香，槍神畫面好屌

遇到問題時除了試著使用其它工具或做法，
還能做的 - 或許也是應該先做的 - 是瞭解問題。

或許你是後端工程師，對前端不熟，那看看你們公司有沒有前端工程師可以問，
不然試著到前端社群上請益看看。
要爬https://www.hktvmall.com/hktv/zh這樣一個網站需要注意什麼？
怎麼爬比較快？

或許有人看了看可以提供一些回饋，
例如有防 csrf 機制，token 就寫在頁面上一個隱藏的 input 裡，https://en.wikipedia.org/wiki/Cross-site_request_forgery會直接跟頁面 html 一起吐回來
後端用的應該是 JAVA，所以 cookie 有個 JSESSIONID，
不過因為它的設定是 http only 無法以 JS 取得
另外本身就神慢，不論是頁面或 API 經常需要 10~30 秒

瞭解了問題之後，或許解法就不會是改用 Selenium 之外的工具，
而是使用 Selenium 但調整做法。
因為就算換了工具慢還是慢在網站本身

例如，一樣先用 Selenium 連到該頁面，
但是不再用操作頁面連結的方式做，而是設法取得 CSRF token 組參數打 api，
使用 execute_script 應該就能在第一次頁面載入後由 input 元素取得 CSRF token，
再使用 execute_async_script 打 api 取得資料
  Ref :https://stackoverflow.com/q/28057338/1042731而由於是實際跑在瀏覽器，可以正確的帶上 JSESSION ID，
並且瀏覽器本身的同 dom

In [9]:
posts_info

{'第 1 篇': {'標題': '[心得] Google裁員1.2萬人只寄電郵',
  '作者': 'Lordaeron',
  '發文日期': '1/26',
  '推文數': '16',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1674717939.A.DDB.html',
  '內文': '來源https://udn.com/news/story/7086/6930865?from=udn_ch2_menu_v2_main_indexGoogle裁員1.2萬人只寄電郵 20年工程師「直接被封鎖」\n在Google擔任研究員的拉塞爾（Dan Russell）則說，「當我凌晨4點完成一項重要分析，\n我要上班時發現鑰匙卡壞了，我在Google工作17年半。\n\n\n這就是台灣公司和google 的實力的差異了。\n不用交接!! 一家可以直接吊打全台灣所有公司。\n\n--'},
 '第 2 篇': {'標題': 'Re: 請益Request爬蟲',
  '作者': 'lovdkkkk',
  '發文日期': '1/27',
  '推文數': '1',
  '文章超連結': 'https://www.ptt.cc/bbs/Soft_Job/M.1674828041.A.5EB.html',
  '內文': '追了一天動漫來回點東西，\n轉生公主天才千金好香，槍神畫面好屌\n\n遇到問題時除了試著使用其它工具或做法，\n還能做的 - 或許也是應該先做的 - 是瞭解問題。\n\n或許你是後端工程師，對前端不熟，那看看你們公司有沒有前端工程師可以問，\n不然試著到前端社群上請益看看。\n要爬https://www.hktvmall.com/hktv/zh這樣一個網站需要注意什麼？\n怎麼爬比較快？\n\n或許有人看了看可以提供一些回饋，\n例如有防 csrf 機制，token 就寫在頁面上一個隱藏的 input 裡，https://en.wikipedia.org/wiki/Cross-site_request_forgery會直接跟頁面 html 一起吐回來\n後端用的應該是 JAVA，所以 cookie 有個 JSESSIONID，\n不過因為它的設定是 http only 無法以 JS 取得\n另外本身就神慢，

<font color= #0fe9de size= 5 face="標楷體"> 將爬取資料轉成 DataFrame </font>

In [10]:
article_df = pd.DataFrame(posts_info)
posts_df = article_df.T
print(posts_df.info())
posts_df

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 第 1 篇 to 第 200 篇
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   標題      200 non-null    object
 1   作者      200 non-null    object
 2   發文日期    200 non-null    object
 3   推文數     200 non-null    object
 4   文章超連結   200 non-null    object
 5   內文      200 non-null    object
dtypes: object(6)
memory usage: 10.9+ KB
None


,標題,作者,發文日期,推文數,文章超連結,內文
第 1 篇,[心得] Google裁員1.2萬人只寄電郵,Lordaeron,1/26,16,https://www.ptt.cc/bbs/Soft_Job/M.1674717939.A...,來源https://udn.com/news/story/7086/6930865?from...
第 2 篇,Re: 請益Request爬蟲,lovdkkkk,1/27,1,https://www.ptt.cc/bbs/Soft_Job/M.1674828041.A...,追了一天動漫來回點東西，\n轉生公主天才千金好香，槍神畫面好屌\n\n遇到問題時除了試著使用...
第 3 篇,[請益] 確保在剛好5分鐘不差作答網頁測驗,freebug,1/28,33,https://www.ptt.cc/bbs/Soft_Job/M.1674909243.A...,我最近在開發一個「線上測驗」的網頁\n\n前端配合後端PHP\n\n線上測驗的系統要剛好在5...
第 4 篇,[請益] 兩份 offer 請益,y2468101216,1/29,18,https://www.ptt.cc/bbs/Soft_Job/M.1674982234.A...,1. 現職 110k/m * 12 灰色產業，主要做 PHP 後端為主，會刻一點後台前端\n...
第 5 篇,Re: [請益] 確保在剛好5分鐘不差作答網頁測驗,wt,1/29,,https://www.ptt.cc/bbs/Soft_Job/M.1674995504.A...,看起來目的是要考試防弊，選擇的方法是透過timestamp。\n只透過timestamp會有...
...,...,...,...,...,...,...
第 196 篇,Re: [討論] 用AI寫code產生的疑問,givemoney,4/03,6,https://www.ptt.cc/bbs/Soft_Job/M.1680497919.A...,我和同事們不久前有討論過如何透過 ChatGPT 來建立應用或是優化工作流\n\n例如協助我...
第 197 篇,Re: [討論] 用AI寫code產生的疑問,pichubaby,4/04,4,https://www.ptt.cc/bbs/Soft_Job/M.1680555333.A...,先講結論，軟體工程師做的事情以及定義從 1946 年 ENIAC 開始就不斷地在改變。\n所...
第 198 篇,[請益] 現在景氣適合轉職嗎?,klolong,4/04,21,https://www.ptt.cc/bbs/Soft_Job/M.1680605272.A...,小弟目前32歲 從事設備工程師 中字輩電類碩畢\n\n因覺得設備這職業前途不佳 萌生轉職軟體...
第 199 篇,Re: [討論] 用AI寫code產生的疑問,ripple0129,4/05,3,https://www.ptt.cc/bbs/Soft_Job/M.1680629761.A...,https://reurl.cc/pLo9ab三星稱10年內SSD達到1PB\n當容量便宜的...


<font color= #0fe9de size= 5 face="標楷體"> 初步檢查 DataFrame </font>

In [11]:
posts_df.describe()

,標題,作者,發文日期,推文數,文章超連結,內文
count,200,200,200,200,200,200
unique,156,159,67,48,200,200
top,Re: [討論] 用AI寫code產生的疑問,applebg,2/24,,https://www.ptt.cc/bbs/Soft_Job/M.1674717939.A...,來源https://udn.com/news/story/7086/6930865?from...
freq,14,6,10,18,1,1


In [12]:
empty = (posts_df == '')    #檢查表格是否有空字串
print(empty.any())

duplicate = posts_df['內文'].duplicated(keep = False) #duplicated() keep參數預設為 True，指標記重複的 row 為 True，第一次出現的 row 不標記。若改為 False 則會將所有重複資料都標記為 True。
duplicate.sum()

標題       False
作者       False
發文日期     False
推文數       True
文章超連結    False
內文       False
dtype: bool


0

<font color= #0fe9de size= 5 face="標楷體"> 儲存資料 </font>

In [13]:
current_dir = os.getcwd()  #利用 os 套件顯示目前資料夾
print(current_dir)
posts_df.to_csv(f'{current_dir}/Soft_Job.csv', index = False, encoding = "utf-8-sig")

C:\Users\yuniy\Desktop\New pioneers in the industry\Self-studying\程式碼練習\爬蟲


In [14]:
#過濾感興趣的文章主題
Soft_Job_KWord = posts_df.loc[posts_df['標題'].str.contains("轉職" or "新鮮人" or "非本科")]
Soft_Job_KWord.to_csv('Soft_Job_KeyWord.csv', index = False, encoding = "utf-8-sig")
print(Soft_Job_KWord.info())
Soft_Job_KWord

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 第 16 篇 to 第 198 篇
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   標題      12 non-null     object
 1   作者      12 non-null     object
 2   發文日期    12 non-null     object
 3   推文數     12 non-null     object
 4   文章超連結   12 non-null     object
 5   內文      12 non-null     object
dtypes: object(6)
memory usage: 672.0+ bytes
None


,標題,作者,發文日期,推文數,文章超連結,內文
第 16 篇,[心得] 非本科轉職軟體工程師的勸退文,gobears5566,2/04,49,https://www.ptt.cc/bbs/Soft_Job/M.1675498189.A...,最近看到新聞提到，今年的年後轉職潮是歷年來最大的一波，人力銀行調查報告指出轉職\n科技業與工...
第 18 篇,Re: [心得] 非本科轉職軟體工程師的勸退文,HZYSoft,2/05,48,https://www.ptt.cc/bbs/Soft_Job/M.1675577047.A...,感謝你的經驗分享，我也是非本科轉職仔，有部分滿同意的你這段講的壓力我很有同感，只是這個問題跟...
第 24 篇,[討論] 27歲非本科轉職來得及嗎？,Loquiall,2/07,57,https://www.ptt.cc/bbs/Soft_Job/M.1675746966.A...,一直以來都對App開發很感興趣的，也持續有透過網路課程學習了解相關知識\n自費上完幾個平台的...
第 25 篇,Re: [討論] 27歲非本科轉職來得及嗎？,applebg,2/07,,https://www.ptt.cc/bbs/Soft_Job/M.1675758620.A...,親，我三十幾歲了，最近考上資工所，下週就要開學了，準備回去當全職\n學生了呢。我在reddi...
第 26 篇,Re: [討論] 27歲非本科轉職來得及嗎？,purplvampire,2/07,,https://www.ptt.cc/bbs/Soft_Job/M.1675778846.A...,"我4x歐吉桑也有去報名, 昨天面試富邦, 週四要面試台哥,\n本業是MIS在職中(MIS版有..."
第 47 篇,[心得] 國中畢業的轉職仔心得分享,ElegonSin,2/14,爆,https://www.ptt.cc/bbs/Soft_Job/M.1676371047.A...,只有國中學歷的轉職路\n各位前輩大家好\n因為目前專案告一段落 讓我上班時可以來分享一下我轉...
第 50 篇,Re: [心得] 非本科轉職軟體工程師的勸退文,answermangtr,2/15,44,https://www.ptt.cc/bbs/Soft_Job/M.1676430443.A...,本肥宅也算是非本科自學(原理組)\n工作年資約七年\n有鑑於最近一年來面試了不少人\n尤其是...
第 90 篇,[推坑] AppWorks School 免費軟體轉職培訓,ooomillie,3/02,57,https://www.ptt.cc/bbs/Soft_Job/M.1677771344.A...,Soft Job 的大家好，我是 AppWorks School 的 Millie，跟大家分...
第 130 篇,[請益] 非本科系轉職的問題（build school）,munn,3/16,11,https://www.ptt.cc/bbs/Soft_Job/M.1678928363.A...,大家好\n我自己是國立大學醫療相關科系畢業，\n畢業後在原本科系工作10年了\n但薪水沒有未...
第 131 篇,[請益] 轉職 兩份offer選擇,suc12345,3/16,37,https://www.ptt.cc/bbs/Soft_Job/M.1678945383.A...,-背景-\n\n大家好，本魯30y，非相關科系公大肄業，程式基礎是資策時期學的Java\n前...


In [15]:
#過濾推文數高的文章
Soft_Job_HighPushNum = posts_df[posts_df['推文數'] =='爆']
posts_df.to_csv('Soft_Job_HighPushNum.csv', index = False, encoding = "utf-8-sig")

Soft_Job_HighPushNum

,標題,作者,發文日期,推文數,文章超連結,內文
第 27 篇,[心得] 我不小心害我徒弟的考績不好,deepcolor,2/07,爆,https://www.ptt.cc/bbs/Soft_Job/M.1675783165.A...,先說我是系統廠 設計驗証部門\n先前十年，一直是孤軍奮戰 業務獨立其他同仁\n專門在幫公司的...
第 47 篇,[心得] 國中畢業的轉職仔心得分享,ElegonSin,2/14,爆,https://www.ptt.cc/bbs/Soft_Job/M.1676371047.A...,只有國中學歷的轉職路\n各位前輩大家好\n因為目前專案告一段落 讓我上班時可以來分享一下我轉...
